<b><span style="color:#E04728;font-size:26pt">Simulation mehrerer Planeten</span></b>

Dieses Notebook simuliert die Bewegung des Merkurs und der Venus zusammen

# Einrichtung der Simulation

Lade alle nötigen Funktionen und richte Grafikausgabe ein.

In [ ]:
from graphics import *

Definiere physikalische Parameter. Die folgenden Parameter wurden berechnet mittels https://nssdc.gsfc.nasa.gov/planetary/factsheet

In [ ]:
rM0 = 4.60     # Startradius des Merkurorbits
vM0 = 5.10e-1  # Startgeschwindigkeit des Merkurs
rV0 = 10.7     # Startradius des Venusorbits
vV0 = 3.05e-1  # Startgeschwindigkeit der Venus
c_aS = 9.90e-1  # Basisbeschleunigung von der Gravitation der Sonne
c_aM = 1.64e-7  # Basisbeschleunigung von der Gravitation des Merkurs
c_aV = 1.93e-7  # Basisbeschleunigung von der Gravitation der Venus
rS  = 2.95e-7  # Schwarzschildradius der Sonne
rL2 = 8.19e-7  # Spezifischer Drehimpuls

Für die physikalischen Berechnungen und grafische Ausgabe brauchen wir Vektoren. Die Startposition und -geschwindigkeit der Planeten können wie folgt definiert werden.

In [ ]:
vec_rM0 = vector(0, rM0, 0) # Startposition des Merkurs
vec_vM0 = vector(vM0, 0, 0) # Startgeschwindigkeit des Merkurs
vec_rV0 = vector(0, rV0, 0) # Startposition der Venus
vec_vV0 = vector(vV0, 0, 0) # Startgeschwindigkeit der Venus

Definiere Simulationsparameter. Diese werden nur von der numerischen Simulation gebraucht und haben keine physikalische Bedeutung.

In [ ]:
dt = 2. * vM0 / 0.99 / 20  # Zeitschritt TODO
max_time = 10000*dt       # Maximale Simulationszeit

# Simuliere die Orbits

In diesem Abschnitt werden die Orbits von Merkur und Venus simuliert.
Die unten definierte Funktion entwickelt einen beliebigen Planeten um einen Schritt in der Zeit.

In [ ]:
def evolve_planet(vec_r_old, vec_v_old, vec_r_other, c_a_other, a, b):
    """
    Berechne einen Zeitschritt eines Planeten.
    Argumente:
         - vec_r_old: alter Positionsvektor des Planeten
         - vec_v_old: alter Geschwindigkeitsvektor des Planeten
         - vec_r_other: Positionsvektor des anderen Planeten
         - c_a_other:sfaktor Beschleunigung auf diesen Planeten die vom anderen Planeten ausgeht
         - a: Stärke des 1/r**3 Terms in der Kraft
         - b: Stärke des 1/r**4 Terms in der Kraft
    Rückgabe:
         - vec_r_new: neuer Positionsvektor des Planeten
         - vec_v_new: neuer Geschwindigkeitsvektor des Planeten
    """

    # Berechne die absolute Beschleunigung basierend auf der Sonne
    aPS = c_aS / vec_r_old.mag**2
    # Multipliziere mit Faktor aus der Allgemeinen Relativitätstheorie
    aPS = aPS * (1 + a * rS / vec_r_old.mag + b * rL2 / vec_r_old.mag**2)
    # Multipliziere mit der Richtung um den Beschleunigungsvektor zu erhalten
    vec_aPS = - aPS * (vec_r_old / vec_r_old.mag)
    
    # Berechne den Vektor vom anderen Planeten zu Diesem
    vec_r_PP = vec_r_old - vec_r_other
    # Berechne die absolute Beschleunigung basierend auf dem anderen Planeten
    aPP = c_a_other / vec_r_PP.mag**2
    # Multipliziere mit der Richtung um den Beschleunigungsvektor zu erhalten
    vec_aPP = - aPP * (vec_r_PP / vec_r_PP.mag)
    
    # Aktualisiere Geschwindigkeitsvektor
    vec_v_new = vec_v_old + (vec_aPS + vec_aPP) * dt
    # Aktualisiere Positionsvektor
    vec_r_new = vec_r_old + vec_v_new * dt
    
    return vec_r_new, vec_v_new

Die folgenden Parameter bestimmen die Stärke der relativistischen Kraft, siehe Poster.
Hier müssen sehr große Werte gewählt werden, damit in der Simulation etwas zu sehen ist.

In [ ]:
a = 0  # Stärke des 1/r**3 tTerms
b = 0  # Stärke des 1/r**4 Terms

In [ ]:
# Erzeuge eine neue Grafikausgabe weiter unten
canvas()

# Zeichne Kugeln and den Startpositionen; M = Merkur, V = Venus, S = Sonne
M = sphere(pos=vec_rM0, radius=0.5, color=color.red)
V = sphere(pos=vec_rV0, radius=0.7, color=color.orange)
S = sphere(pos=vector(0, 0, 0), radius=1.5, color=color.yellow)
# Setze Startgeschwindigkeiten
M.velocity = vec_vM0
V.velocity = vec_vV0
S.velocity = vector(0, 0, 0)

# Mache die Flugbahn der Planeten sichtbar
M.trajectory = curve(color=color.black, radius=0.02)
V.trajectory = curve(color=color.black, radius=0.02)

# Führe die Simulation für die gegebene Zeit aus und zeichne die Trajektorie
time = 0  # Aktuelle Simulationszeit
while time < max_time:
    # Setzte Bildrate
    rate(200)
    # Füge die aktuellen Positionen zu den Trajektorien hinzu
    M.trajectory.append(pos=M.pos)
    V.trajectory.append(pos=V.pos)
    # Aktualisiere Positionen und Geschwindigkeiten
    M.pos, M.velocity = evolve_planet(M.pos, M.velocity, V.pos, c_aV, a, b)
    V.pos, V.velocity = evolve_planet(V.pos, V.velocity, M.pos, c_aM, 0, 0)
    
    # Setze Simulationszeit weiter
    time = time + dt